In [ ]:
!pip install ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! sudo apt-get install -y pciutils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (307 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

In [ ]:
! curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10091    0 10091    0     0  33914      0 --:--:-- --:--:-- --:--:-- 33976
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])


ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [ ]:
! ollama run llama3

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB         

In [ ]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [ ]:
import pandas as pd
import requests
import json

# File paths
train_df_file = "/content/drive/My Drive/train_sample.csv"

train_df = pd.read_csv(train_df_file)

train_df['label#'] = train_df['label'].apply(lambda x: 1 if x == 'dga' else 0)

print(len(train_df))
print(train_df)

2021
                      domain      family   label  label#
0                   stbid.ru       legit  notdga       0
1             xonzhsspmk.com  dnschanger     dga       1
2               apple.com.cn       legit  notdga       0
3                addthis.com       legit  notdga       0
4               autodesk.com       legit  notdga       0
...                      ...         ...     ...     ...
2016   gakepadainqu.ddns.net       symmi     dga       1
2017     duzgvnrcldjgmmj.com       bedep     dga       1
2018              chinaz.com       legit  notdga       0
2019             jxsaumg.com       vidro     dga       1
2020  vutiqoxiokivq.ddns.net       symmi     dga       1

[2021 rows x 4 columns]


In [ ]:
import requests
import json
import pandas as pd

def classify_domain(domain, url='http://localhost:11434/api/chat'):

    # Create the introductory statement
    intro = (
        "You are a domain name classification system. Your task is to classify domain names as either 'dga' (Domain Generation Algorithm) or 'normal'. "
        "DGA domains are automatically generated by malware, while normal domains are not. "
        "I will provide you with labeled training data containing domain names and their classifications. "
        "After the training phase, you will classify a new domain and respond with either 'dga' or 'normal'. \n\n "

    )

    # Add labeled examples from the DataFrame
    prompt = intro
    # Shuffle the DataFrame to ensure randomness
    for _, row in train_df.iterrows():
        label = "dga" if row['label#'] == 1 else "normal"
        prompt += f"{row['domain']}\n{{ 'domain': '{row['domain']}', 'result': '{label}' }}\n\n"

    # Add the new domain to classify
    prompt += f"{{ 'Now you classify this domain': '{domain}', 'only answer dga or normal, Do not provide any additional information or explanation ': "

    # Send the prompt to the model
    question = prompt + "}}"



    payload = {
        "model": "llama3",
        "temperature": 0.6,
        "stream": False,
        "messages": [
            {"role": "system", "content": "You are an AI assistant!"},
            {"role": "user", "content": question}
        ]
    }
    response = requests.post(url, json=payload)
    message_str = response.content.decode('utf-8')
    message_dict = json.loads(message_str)
    final_classification = message_dict['message']['content']

    return final_classification




# Función para convertir las clasificaciones a 1 o 0
def convert_classification(row):
    # Convert the row to lowercase to ensure case-insensitive matching
    row = row.lower()

    # Use regular expressions to find exact matches for 'normal' and 'dga'
    if re.search(r'\bnormal\b', row):
        if re.search(r'\bdga\b', row):
            # Handle cases where both 'normal' and 'dga' appear in the response
            if row.index('normal') < row.index('dga'):
                return 0  # Assume the first occurrence is more likely the intended classification
            else:
                return 1
        return 0
    elif re.search(r'\bdga\b', row):
        return 1
    else:
        # Handle unexpected cases, you might want to log these for further analysis
        print(f"Unexpected response format: {row}")
        return -1  # Or handle in a way appropriate to your use case



# Test with different sizes of training data
training_sizes = [2000]



for size in training_sizes:


    # Example usage with classification
    print(f"Testing with training size: {size}")
    domains = ["as.com", "youtube.com"]
    for domain in domains:
        print(f"Domain: {domain}, Response: {classify_domain(domain)}")

    # Apply classify_domain function to each domain in test_df and store the results in a new column
    #test_df['classification'] = test_df['domain'].apply(lambda domain: classify_domain(domain, train_df, reasoning_output, train_df_reason))

    # Save the classification results
    #test_results_file = f'test_results_{size}.csv'
    #test_df.to_csv(f'/content/drive/My Drive/{test_results_file}', index=False)


    #print(f"Test results saved to {test_results_file}")


Testing with training size: 2000
Domain: as.com, Response: { 'domain': 'as.com', 'result': 'normal' }
Domain: youtube.com, Response: { 'domain': 'youtube.com', 'result': 'normal' }


In [ ]:
import requests
import pandas as pd
import numpy as np
import time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import sys
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import re

families = [

 'alureon.gz',
 'bamital.gz'#,
 #'banjori.gz',
 #'bedep.gz',
 #'charbot.gz',
 #'chinad.gz',
 #'conficker.gz',
 #'corebot.gz',
 #'cryptolocker.gz',
 #'deception.gz',
 #'dircrypt.gz',
 #'dnschanger.gz',
 #'dyre.gz',
 #'emotet.gz',
 #'fobber.gz',
 #'gameover.gz',
 #'gozi.gz',
 #'kraken.gz',
 #'legit.gz',
 #'locky.gz',
 #'manuelita.gz',
 #'matsnu.gz',
 #'monerominer.gz',
 #'murofet.gz',
 #'murofetweekly.gz',
 #'mydoom.gz',
 #'necurs.gz',
 #'nymaim.gz',
 #'oderoor.gz',
 #'padcrypt.gz',
 #'pitou.gz',
 #'proslikefan.gz',
 #'pushdo.gz',
 #'pykspa.gz',
 #'qadars.gz',
 #'qakbot.gz',
 #'qsnatch.gz',
 #'ramdo.gz',
 #'ramnit.gz',
 #'ranbyus.gz',
 #'rovnix.gz',
 #'shiotob.gz',
 #'simda.gz',
 #'sisron.gz',
 #'sphinx.gz',
 #'suppobox.gz',
 #'symmi.gz',
 #'tempedreve.gz',
 #'tinba.gz',
 #'tinynuke.gz',
 #'vawtrak.gz',
 #'vidro.gz',
 #'virut.gz',
 #'zeus-newgoz.gz',

 #'zloader.gz'
 ]

runs = 30
for family in families:
    print(family)
    dga = pd.read_csv(f'/content/drive/My Drive/Familias_Test/{family}', chunksize=50)
    legit = pd.read_csv('/content/drive/My Drive/Familias_Test/legit.gz', chunksize=50)
    dfs = []
    for run in range(runs):
        print(f'{run:2}/{runs}', end='\r')
        dfw = pd.concat([dga.get_chunk(),legit.get_chunk()])
        pred = []
        prob = []
        query_time = []
        results = []

        for domain_to_check in dfw.domain.values:
            st = time.time()

            # Apply classify_domain function to each domain in test_df and store the results in a new column
            #response_model = classify_domain(domain_to_check, train_df, reasoning_output, train_df_reason))
            # Llamada correcta a la función sin el paréntesis adicional
            response_model = classify_domain(domain_to_check)

            # Convertir la columna de clasificación a 1 o 0
            label_value =convert_classification(response_model)


            # Append the result to the list
            pred.append(label_value)
            query_time.append(time.time() - st)
            # Append the generated prompt to the list
        #generated_prompts.append(generated_text)

        dfw['pred'] = pred
        #df['prob'] = prob
        dfw['query_time'] = query_time
        dfw.to_csv(f'/content/drive/My Drive/results/results_Llama3_docker_{family}_{run}.csv.gz', index=False)

bamital.gz
Unexpected response format: 'norm
Unexpected response format: you are an ai assistant!
Unexpected response format: you are an ai assistant!
Unexpected response format: you are an ai assistant!
Unexpected response format: you are an ai assistant!
Unexpected response format: you are an ai assistant!
Unexpected response format: you are an ai assistant!


In [ ]:
families = [

 'alureon.gz',
 'bamital.gz']
def false_positive_rate(y, ypred):
    P = (y == 1).sum()
    N = (y == 0).sum()
    TP = np.logical_and(y == ypred, y == 1).sum()
    TN = np.logical_and(y == ypred, y == 0).sum()
    FP = np.logical_and(y != ypred, y == 1).sum()
    FN = np.logical_and(y != ypred, y == 0).sum()
    return FP/N

for family in families:
    acc = []
    pre = []
    rec = []
    f1 = []
    fpr = []
    qt = []
    qts = []
    for run in range(runs):
        dfw = pd.read_csv(f'/content/drive/My Drive/results/results_Llama3_docker_{family}_{run}.csv.gz')
        dfw = dfw[dfw.pred != -1]
        y = (dfw.label == 'dga').astype(int)
        #ypred = (dfw.pred == 'dga').astype(int)
        ypred = dfw.pred
        acc.append(accuracy_score(y, ypred))
        pre.append(precision_score(y, ypred))
        rec.append(recall_score(y, ypred))
        f1.append(f1_score(y, ypred))
        fpr.append(false_positive_rate(y, ypred))
        qt.append(dfw.query_time.mean())
        qts.append(dfw.query_time.std())
#    print(f'Query time: {np.mean(qt):0.5f}+/-{np.mean(qts)}:0.5f')
    print(f'{family.split(".")[0]:15}: acc:{np.mean(acc):0.2f}±{np.std(acc):.3f} f1:{np.mean(f1):0.2f}±{np.std(f1):.3f} pre:{np.mean(pre):0.2f}±{np.std(pre):.3f} rec:{np.mean(rec):0.2f}±{np.std(rec):.3f}  FPR:{np.mean(fpr):0.2f}±{np.std(fpr):.3f}  Query time: {np.mean(qt):0.5f}±{np.mean(qts):0.5f}')

alureon        : acc:0.92±0.027 f1:0.93±0.027 pre:0.91±0.032 rec:0.95±0.033  FPR:0.05±0.033  Query time: 1.55919±0.25988
bamital        : acc:0.91±0.028 f1:0.91±0.029 pre:0.90±0.032 rec:0.92±0.047  FPR:0.08±0.047  Query time: 1.77842±0.36450
